In [142]:
import pandas as pd
from census_api import census_df
from scrape_realtor import realtor_df

In [143]:
state_abrevs=pd.read_json('state_abrev.json', dtype="str")
state_abrevs

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA
5,Colorado,Colo.,CO
6,Connecticut,Conn.,CT
7,Delaware,Del.,DE
8,District of Columbia,D.C.,DC
9,Florida,Fla.,FL


In [144]:
realtor_df["State"] = realtor_df["State"].str.upper().str.replace(" ", "")
realtor_df["County"] = realtor_df["County"].str.capitalize()
realtor_df.head()



,County,State,month_date_yyyymm,county_fips,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,pending_ratio,quality_flag,Code
0,Burleigh,ND,202208,38015,382450,225,39,136,0,80,238,170,2212,464139,464,1.0578,0,ND
1,Walthall,MS,202208,28147,169250,13,51,4,0,4,2,92,2181,251766,15,0.1538,1,MS
2,Pulaski,IL,202208,17153,84700,9,67,4,0,4,4,62,1474,87918,13,0.4444,0,IL
3,Petersburg city,VA,202208,51730,194995,117,36,84,4,60,125,140,1406,217184,243,1.0684,1,VA
4,Houghton,MI,202208,26061,254450,50,37,36,0,16,42,122,1761,352869,95,0.84,1,MI


In [145]:
realtor_df= pd.merge(realtor_df, state_abrevs, how='left', left_on="State", right_on="Code")
realtor_df


,County,State_x,month_date_yyyymm,county_fips,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,...,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,pending_ratio,quality_flag,Code_x,State_y,Abbrev,Code_y
0,Burleigh,ND,202208,38015,382450,225,39,136,0,80,...,170,2212,464139,464,1.0578,0,ND,North Dakota,N.D.,ND
1,Walthall,MS,202208,28147,169250,13,51,4,0,4,...,92,2181,251766,15,0.1538,1,MS,Mississippi,Miss.,MS
2,Pulaski,IL,202208,17153,84700,9,67,4,0,4,...,62,1474,87918,13,0.4444,0,IL,Illinois,Ill.,IL
3,Petersburg city,VA,202208,51730,194995,117,36,84,4,60,...,140,1406,217184,243,1.0684,1,VA,Virginia,Va.,VA
4,Houghton,MI,202208,26061,254450,50,37,36,0,16,...,122,1761,352869,95,0.84,1,MI,Michigan,Mich.,MI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,Oneida,ID,202208,16071,279900,26,59,8,0,12,...,159,2110,423895,37,0.3462,1,ID,Idaho,Idaho,ID
3102,St. joseph,MI,202208,26149,232000,103,42,64,0,52,...,128,1850,305506,204,0.9806,0,MI,Michigan,Mich.,MI
3103,Wheeler,TX,202208,48483,139900,20,45,8,0,0,...,99,1414,139790,29,0.45,0,TX,Texas,Tex.,TX
3104,Christian,KY,202208,21047,195000,85,32,80,0,36,...,126,1599,217868,195,1.2941,0,KY,Kentucky,Ky.,KY


In [146]:
census_df['County'] = census_df['County'].str.replace("County", "")

print(census_df.head())


     County     State  Total:  Total:Population of one race:  \
0  Autauga    Alabama   58805                          55648   
1  Baldwin    Alabama  231767                         216743   
2  Barbour    Alabama   25223                          24523   
3     Bibb    Alabama   22293                          21534   
4   Blount    Alabama   59134                          55478   

   Total:Population of one race:White alone  \
0                                     42160   
1                                    189399   
2                                     11317   
3                                     16555   
4                                     50663   

   Total:Population of one race:Black or African American alone  \
0                                              11445              
1                                              18217              
2                                              11933              
3                                               4413            

In [152]:
from sqlalchemy import create_engine 
from postgres_credentials import aclhemycred

engine= create_engine(alchemycred)

ModuleNotFoundError: No module named 'postgres_credentials'

In [149]:
realtor_df.to_sql("county_prices", engine)

OperationalError: (psycopg2.OperationalError) FATAL:  role "username" does not exist

(Background on this error at: https://sqlalche.me/e/14/e3q8)